In [1]:
# Quantized LLama
from ctransformers import AutoModelForCausalLM,AutoTokenizer

# Embedding Model
from sentence_transformers import SentenceTransformer, util
from sentence_transformers.util import cos_sim

# Kmeans and PCA 
from sklearn.cluster import KMeans
from sklearn.metrics import silhouette_samples, silhouette_score
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA

# Plotting
import matplotlib.pyplot as plt

# NLP utilities
import re
import nltk
from nltk.corpus import stopwords
from nltk.util import ngrams 
import string

# Python dataframes and arrays
import pandas as pd
import numpy as np

# Misc
from collections import Counter

import requests
import json
import torch
from ctransformers import AutoModelForCausalLM
import random
import pandas as pd

from gensim import corpora
from gensim.models import LdaModel
from gensim.models.coherencemodel import CoherenceModel
from pprint import pprint


from sklearn.feature_extraction.text import CountVectorizer
from sklearn.decomposition import LatentDirichletAllocation

In [2]:
## Set up llama on Azure
target_url = 'https://Llama-2-70b-chat-dft-serverless.eastus2.inference.ai.azure.com'
api_key = 't8FlgEG09Wlr9MfgXeIDMkJy0kv5eb8R'

def ask_llama(prompt, temp = 1, max_tokens = 10000):
    llm_inputs = {
            "messages":
            [
                { 
                "role": "system", 
                "content": prompt
                }
            ],
            "temperature": temp,
        }
    response = requests.post(f'{target_url}/v1/chat/completions', headers={'Authorization': f'Bearer {api_key}', 'Content-Type': 'application/json'}, data=json.dumps(llm_inputs))
    return(response.json()['choices'][0]['message']['content'])

In [14]:
#Read data and preprocess it
df = pd.read_csv("../Data/cleaned_analytic_file_for_viz_290224.csv")
## Seperate responses who mention not_know to make processing faster
question_scenario_one = "You are using an AI clinical decision-support system which recommends \
treatment options for individual patients. You disagree with the recommendation that is given by \
the system. How would you proceed?"

not_know_lists = ["dont know","don't know", "not know", "not sure", "don’t know"]
df = df.dropna(subset=['scenario_one'])
df['not_know'] = df['scenario_one'].apply(lambda x: 1 if any(ss.lower() in x.lower() for ss in not_know_lists) else 0)
df_filtered = df[df['not_know']== 0]
df_filtered.index = range(len(df_filtered)) #reindex the new list to start from 0-len. 

### Using Gensim to extract topics from each response. 

[(0,
  '0.100*"I will check national and local guidelines before follow AI recommendations or my own decision.\r\r\n" + 0.100*"I would choose the treatment option based on the national/local guidelines and justify these with reasoning." + 0.100*"I would prescribe based on my clinical training and standard guidelines/evidence" + 0.100*"Patient management is multifaceted and usually takes into consideration features from the history, examination and investigation findings to decide if treatment is indicated in a patient. I am less likely to be inflexible in my assessment  and decision to treat a patient than an AI." + 0.100*"It is a supportive system and not an alternative to the human. So I would override it and continue on my own" + 0.100*"get help from a senior doctor or colleague" + 0.100*"Recheck guideline parameters\r\r\nAsk peers for advice\r\r\nEscalate to senior clinician \r\r\n" + 0.100*"I would seek other opinions from my colleagues or experts in the field. I would be happy to